In [9]:
from __future__ import print_function
#%matplotlib inline
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torchsummary import summary
from sklearn.metrics import accuracy_score
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from image_folder_redesign import ImageFolder

# Set random seed for reproducibility
manualSeed = 999
#manualSeed = random.randint(1, 10000) # use if you want new results
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)

Random Seed:  999


In [10]:
dataset_path = './Datasets/ModelNet10Voxel/'
train_path = dataset_path + 'Train/'
test_path = dataset_path + 'Test/'
all_path = dataset_path + 'All/'

# Number of workers for dataloader
workers = 2

# Batch size during training
batch_size = 100

# Spatial size of training images. All images will be resized to this
#   size using a transformer.
image_size = 64

# Number of channels in the training images. For color images this is 3
nc = 1

# Size of z latent vector (i.e. size of generator input)
nz = 200

# Size of feature maps in generator
ngf = 64

# Size of feature maps in discriminator
ndf = 64

# Number of training epochs
num_epochs = 200

# Learning rate for optimizers
lr_g = 0.000025
lr_d = 0.0000001

# Beta1 hyperparam for Adam optimizers
beta1 = 0.5

# Number of GPUs available. Use 0 for CPU mode.
ngpu = 1

# Decide which device we want to run on
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

In [15]:
# custom weights initialization called on netG and netD
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

In [16]:
# Generator Code

class Generator(nn.Module):
    def __init__(self, ngpu):
        super(Generator, self).__init__()
        self.ngpu = ngpu
        # ConvTranspose3d(in_channels, out_channels, kernel_size, 
        # stride=1, padding=0, output_padding=0, groups=1, 
        # bias=True, dilation=1, padding_mode='zeros')
            
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose3d(nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm3d(ngf * 8),
            nn.ReLU(inplace=True),
            
            nn.ConvTranspose3d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm3d(ngf * 4),
            nn.ReLU(inplace=True),

            nn.ConvTranspose3d( ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm3d(ngf * 2),
            nn.ReLU(inplace=True),
            
            nn.ConvTranspose3d( ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm3d(ngf),
            nn.ReLU(inplace=True),
            
            nn.ConvTranspose3d(ngf, nc, 4, 2, 1, bias=False),
            nn.Sigmoid()
            
        )

    def forward(self, input):
        return self.main(input)

In [17]:
# Create the generator
netG = Generator(ngpu).to(device)

# Handle multi-gpu if desired
if (device.type == 'cuda') and (ngpu > 1):
    netG = nn.DataParallel(netG, list(range(ngpu)))

# Apply the weights_init function to randomly initialize all weights
#  to mean=0, stdev=0.2.
netG.apply(weights_init)

# Print the model
print(summary(netG, input_size=(200,1,1,1)))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
   ConvTranspose3d-1         [-1, 512, 4, 4, 4]       6,553,600
       BatchNorm3d-2         [-1, 512, 4, 4, 4]           1,024
              ReLU-3         [-1, 512, 4, 4, 4]               0
   ConvTranspose3d-4         [-1, 256, 8, 8, 8]       8,388,608
       BatchNorm3d-5         [-1, 256, 8, 8, 8]             512
              ReLU-6         [-1, 256, 8, 8, 8]               0
   ConvTranspose3d-7      [-1, 128, 16, 16, 16]       2,097,152
       BatchNorm3d-8      [-1, 128, 16, 16, 16]             256
              ReLU-9      [-1, 128, 16, 16, 16]               0
  ConvTranspose3d-10       [-1, 64, 32, 32, 32]         524,288
      BatchNorm3d-11       [-1, 64, 32, 32, 32]             128
             ReLU-12       [-1, 64, 32, 32, 32]               0
  ConvTranspose3d-13        [-1, 1, 64, 64, 64]           4,096
          Sigmoid-14        [-1, 1, 64,

In [18]:
class Discriminator(nn.Module):
    def __init__(self, ngpu):
        super(Discriminator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(

            nn.Conv3d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv3d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm3d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv3d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm3d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv3d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm3d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv3d(ndf * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.main(input)

In [20]:
# Create the Discriminator
netD = Discriminator(ngpu).to(device)

# Handle multi-gpu if desired
if (device.type == 'cuda') and (ngpu > 1):
    netD = nn.DataParallel(netD, list(range(ngpu)))

# Apply the weights_init function to randomly initialize all weights
#  to mean=0, stdev=0.2.
netD.apply(weights_init)

# Print the model
summary(netD, input_size=(1,64,64,64))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv3d-1       [-1, 64, 32, 32, 32]           4,096
         LeakyReLU-2       [-1, 64, 32, 32, 32]               0
            Conv3d-3      [-1, 128, 16, 16, 16]         524,288
       BatchNorm3d-4      [-1, 128, 16, 16, 16]             256
         LeakyReLU-5      [-1, 128, 16, 16, 16]               0
            Conv3d-6         [-1, 256, 8, 8, 8]       2,097,152
       BatchNorm3d-7         [-1, 256, 8, 8, 8]             512
         LeakyReLU-8         [-1, 256, 8, 8, 8]               0
            Conv3d-9         [-1, 512, 4, 4, 4]       8,388,608
      BatchNorm3d-10         [-1, 512, 4, 4, 4]           1,024
        LeakyReLU-11         [-1, 512, 4, 4, 4]               0
           Conv3d-12           [-1, 1, 1, 1, 1]          32,768
          Sigmoid-13           [-1, 1, 1, 1, 1]               0
Total params: 11,048,704
Trainable para

In [22]:
# Create dataloader
data_transforms = transforms.Compose([
            transforms.ToTensor()
        ])

root = "/home/mvries/Documents/Datasets/ModelNet10Voxel/"
dataset = ImageFolder(root=root+"Train/", transform=data_transforms, size=64)

dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                         shuffle=True, num_workers=workers)

In [23]:
from torch.optim import lr_scheduler

# Initialize BCELoss function
criterion = nn.BCELoss()

# Create batch of latent vectors that we will use to visualize
#  the progression of the generator
fixed_noise = torch.randn(1, nz, 1, 1, 1)

# Establish convention for real and fake labels during training
real_label = 1.
fake_label = 0.

# Discriminator learns faster than the generator
# For each batch, the discriminator only gets updated if the discriminator only 
# gets updated if its accuracy in the last batch is not higher than 80%
lrG = 0.0025
lrD = 0.00001


# Setup Adam optimizers for both G and D
optimizerD = optim.Adam(netD.parameters(), lr=lrD, betas=(beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lrG, betas=(beta1, 0.999))

In [ ]:
# Training Loop

# Lists to keep track of progress
img_list = []
G_losses = []
D_losses = []
iters = 0

acc = 0

print("Starting Training Loop...")
# For each epoch
for epoch in range(100):
    # For each batch in the dataloader
    for i, data in enumerate(dataloader, 0):
        ############################
        # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################
        ## Train with all-real batch
        netD.zero_grad()
        # Format batch
        real_cpu = data[0]
        threshold = 0.0
        real_cpu = (real_cpu > threshold).type(torch.FloatTensor).to(device)
        b_size = real_cpu.size(0)
        label = torch.full((b_size,), real_label, dtype=torch.float, device=device)

        # Forward pass real batch through D

        output = netD(real_cpu).view(-1)
        real_pred = (output> 0.5).type(torch.FloatTensor)
        # Calculate loss on all-real batch
        errD_real = criterion(output, label)
        # Calculate gradients for D in backward pass
        errD_real.backward()
        D_x = output.mean().item()
        
        ## Train with all-fake batch
        ## Generate batch of latent vectors
        noise = torch.randn(b_size, nz, 1, 1, 1, device=device)
        # Generate fake image batch with G
        fake = netG(noise)
        label.fill_(fake_label)

        # Classify all fake batch with D
        ouatput = netD(fake.detach()).view(-1)
        fake_pred = (output > 0.5).type(torch.FloatTensor)
        # Calculate D's loss on the all-fake batch
        errD_fake = criterion(output, label)
        # Calculate the gradients for this batch, accumulated (summed) with previous gradients
        errD_fake.backward()
        D_G_z1 = output.mean().item()
        # Compute error of D as sum over the fake and the real batches
        errD = errD_real + errD_fake
        
        # For each batch, the discriminator only gets updated if the discriminator only 
        # gets updated if its accuracy in the last batch is not higher than 80%
        
        real_lab = torch.ones((b_size,))
        fake_lab = torch.zeros((b_size,))
        
        labs = torch.cat((real_lab, fake_lab))
        pred = torch.cat((real_pred, fake_pred))
        
        if acc <=0.85:
            # Update D
            optimizerD.step()
        
        acc = accuracy_score(labs, pred)

        ############################
        # (2) Update G network: maximize log(D(G(z)))
        ###########################
        netG.zero_grad()
        label.fill_(real_label)  # fake labels are real for generator cost
        # Since we just updated D, perform another forward pass of all-fake batch through D
        output = netD(fake).view(-1)
        # Calculate G's loss based on this output
        errG = criterion(output, label)
        # Calculate gradients for G
        errG.backward()
        D_G_z2 = output.mean().item()
        # Update G
        optimizerG.step()

        # Output training stats
        if i % 10 == 0:
            print('[%d/%d][%d/%d]\tLoss_D: %.8f\tLoss_G: %.8f\tD(x): %.8f\tD(G(z)): %.8f / %.8f'
                  % (epoch, num_epochs, i, len(dataloader),
                     errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))

        # Save Losses for plotting later
        G_losses.append(errG.item())
        D_losses.append(errD.item())

        # Check how the generator is doing by saving G's output on fixed_noise
        if (iters % 10 == 0) or ((epoch == num_epochs-1) and (i == len(dataloader)-1)):
            with torch.no_grad():
                fake = netG(fixed_noise.type(torch.FloatTensor).to(device)).detach().cpu()
                fake = (fake.detach().cpu() > 0.5).type(torch.FloatTensor)
            img_list.append(fake)

        iters += 1

In [ ]:
plt.figure(figsize=(10,5))
plt.title("Generator and Discriminator Loss During Training")
plt.plot(G_losses,label="G")
plt.plot(D_losses,label="D")
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
torch.save({
        'model_state_dict': netG.state_dict(),
        'optimizer_state_dict': optimizerG.state_dict()
    }, 'Generator' + '.pt')

torch.save({
        'model_state_dict': netD.state_dict(),
        'optimizer_state_dict': optimizerD.state_dict()
    }, 'Discriminator' + '.pt')